In [1]:
import sys
sys.path.append('..')

import os

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

In [4]:
ruta = open('../ruta_local.txt')
ruta = ruta.readlines()[0]
ruta

'OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [5]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta_local.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(RUTA_COMPLETA + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(RUTA_COMPLETA + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [6]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

## USUARIOS ACTIVOS

In [11]:
def get_activos():
    query_1 = '''select 
                    DATE(tok.created_at) as created_at, 
                    usu.codigo, 
                    usu.name, 
                    usu.apellido_paterno, 
                    usu.apellido_materno, 
                    usu.username, 
                    usu.tipo, 
                    \'reg acces\' as origen
                from oauth_access_tokens tok
                inner join users usu on tok.user_id=usu.id
                WHERE 
                    tok.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH) and 
                    usu.tipo=\'propietario\''''
    usuarios_acces = dbQuery(query_1)
    data_acces = pd.DataFrame(usuarios_acces)
    
    query_2 = '''select 
                    DATE(ac.created_at) as created_at, 
                    us.codigo, 
                    us.name, 
                    us.apellido_paterno, 
                    us.apellido_materno, 
                    us.username, us.tipo, 
                    \'reg. labor\'  as origen\
                from actividades ac
                inner join users us on us.id = ac.usuario_registro
                WHERE 
                    ac.created_at >= DATE_SUB(NOW(), INTERVAL 6 MONTH)'''
    usuarios_reg = dbQuery(query_2)
    data_reg = pd.DataFrame(usuarios_reg)
    data = pd.concat([data_acces, data_reg], ignore_index=True)
    data.to_excel(RUTA_COMPLETA + '/_agrocittca-data/' + "usuarios_activos.xlsx", index=False)

    
def get_variedades():
    query = 'select * from variedad'
    variedades = dbQuery(query)
    data = pd.DataFrame(variedades)
    data.to_excel(RUTA_COMPLETA + '/_agrocittca-data/' + "variedades.xlsx", index=False)
get_variedades()

In [7]:
periodos = {16, 17}
periodos = "(" + ", ".join(map(str, periodos)) + ")"
periodos

'(16, 17)'

In [8]:
#rops = {1618, 500}
props = {367, 636, 1184, 1314, 500, 1723, 80, 1618, 2075, 628, 949}
props = "(" + ", ".join(map(str, props)) + ")"
props

'(1184, 1314, 367, 80, 1618, 500, 628, 949, 2075, 636, 1723)'

In [9]:
def get_propiedades():
    query = f"select * from propiedad\
                where estado = \'activo\'\
                and periodo_id in {periodos}\
                and codigo in {props}"
    propiedades = dbQuery(query)
    data = pd.DataFrame(propiedades)
    data.to_excel(ruta + '/_agrocittca-data/' + "propiedades.xlsx", index=False)
    return data

def get_periodos(peris):
    query = f"select * from periodo\
            where estado = 'activo'\
            and id in {peris}"
    periodos = dbQuery(query)
    data = pd.DataFrame(periodos)
    data.to_excel(ruta + '/_agrocittca-data/' + "periodos.xlsx", index=False)
    return data

def get_canheros(ca):
    query = f"select * from users\
            where estado = 'activo'\
            and id in {ca}"
    canheros = dbQuery(query)
    data = pd.DataFrame(canheros)
    data.to_excel(ruta + '/_agrocittca-data/' + "canheros.xlsx", index=False)
    return data

def get_lotes(props_ids):
    query = f"select distinct(l.id), l.* from propiedad prop\
                inner join lote l on l.propiedad_id = prop.id\
                inner join actividades ac on ac.lote_id = l.id\
                where l.estado = 'activo'\
                and ac.estado = 'activo'\
                and prop.id in {props_ids}"
    lotes = dbQuery(query)
    data = pd.DataFrame(lotes)
    #data['superficie'] = data['superficie'].str.replace('.', ',')
    data['superficie'] = data['superficie'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "lotes.xlsx", index=False)
    return data

def get_actividades(props_ids):
    query = f"select ac.* from propiedad prop\
                inner join lote l on l.propiedad_id = prop.id\
                inner join actividades ac on ac.lote_id = l.id\
                where l.estado = 'activo'\
                and ac.estado = 'activo'\
                and prop.id in {props_ids}"
    actividades = dbQuery(query)
    data = pd.DataFrame(actividades)
    data['superficie'] = data['superficie'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "actividades.xlsx", index=False)
    return data

def get_tipo_actividad(periodos_ids):
    query = f"select * from tipo_actividad\
                where estado='activo'\
                and periodo_id in {periodos_ids}"
    tipo_ac = dbQuery(query)
    data = pd.DataFrame(tipo_ac)
    data['precio_unitario'] = data['precio_unitario'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "tipo_actividad.xlsx", index=False)
    return data

def get_labor():
    query = f"select * from labor where estado='activo'"
    labor = dbQuery(query)
    data = pd.DataFrame(labor)
    data.to_excel(ruta + '/_agrocittca-data/' + "labor.xlsx", index=False)
    return data

def get_detalle_actividad(periodos_ids):
    query = f"select det_ac.* from detalle_actividad det_ac\
            inner join actividades ac on ac.id = det_ac.actividad_id\
            where det_ac.estado='activo'\
            and ac.periodo_id in {periodos_ids}"
    det_ac = dbQuery(query)
    data = pd.DataFrame(det_ac)
    data['cantidad'] = data['cantidad'].astype(float)
    data['precio_producto'] = data['precio_producto'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "detalle_actividad.xlsx", index=False)
    return data

def get_producto_act():
    query = f"select prod.*, cat_prod.nombre insumo from producto_actividad prod\
                inner join categoria_producto cat_prod on cat_prod.id = prod.categoria_producto_id\
                where prod.estado = 'activo'"
    producto_act = dbQuery(query)
    data = pd.DataFrame(producto_act)
    data['precio_unitario'] = data['precio_unitario'].astype(float)
    data.to_excel(ruta + '/_agrocittca-data/' + "producto_actividad.xlsx", index=False)
    return data

In [10]:
propiedades = get_propiedades()

periodos = set(propiedades['periodo_id'])
periodos = "(" + ", ".join(map(str, periodos)) + ")"
get_periodos(periodos)
periodos

canheros = set(propiedades['usuario_id'])
canheros = "(" + ", ".join(map(str, canheros)) + ")"
get_canheros(canheros)
canheros

props_ids = set(propiedades['id'])
props_ids = "(" + ", ".join(map(str, props_ids)) + ")"
lotes = get_lotes(props_ids)
lotes

actividades = get_actividades(props_ids)
actividades

tipo_actividad = get_tipo_actividad(periodos)
tipo_actividad.head(3)

labor = get_labor()
labor

detalle_ac = get_detalle_actividad(periodos)
detalle_ac.head(3)

producto_act = get_producto_act()
producto_act.head(3)

,id,nombre,descripcion,precio_unitario,ingrediente_activo,dosis_recomendada,estado,slug,unidad_medida_id,categoria_producto_id,created_at,updated_at,insumo
0,1,Sunami,,166.76,FIPRONIL 80%,4,activo,sunami-1,1,1,2022-08-31 17:07:16,2022-08-31 17:07:16,Insecticida
1,3,Merlin Total,U.C.G.,147.50,Merlin Total SC600,0,activo,merlin-total-3,2,3,2022-08-31 17:07:16,2023-02-22 15:48:37,Pre-Emergente
2,4,Flutolex,None,134.00,ISOXAFLUTOLE,0,activo,flutolex-4,2,3,2022-08-31 17:07:16,2023-01-30 09:44:36,Pre-Emergente


In [12]:
get_activos()


In [ ]:
get_actividades()

In [36]:
get_tipo_actividad()

In [38]:
get_labor()

In [42]:
get_maquinaria()

In [44]:
get_operario()

In [46]:
get_categoria_producto()